In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
url='http://books.toscrape.com/'
response=requests.get(url)
response

<Response [200]>

In [ ]:
response.status_code

200

In [ ]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [ ]:
type(response.text)

str

In [ ]:
soup = bs(response.text)
type(soup)


bs4.BeautifulSoup

In [ ]:
soup.find('title').text

'\n    All products | Books to Scrape - Sandbox\n'

In [ ]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [ ]:
books_tag = soup.find_all('article', class_='product_pod')

In [ ]:
len(books_tag)

20

In [ ]:
book_tag=books_tag[0]
book_tag

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [ ]:
title_tag = book_tag.find('a', title=True)
title_tag

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [ ]:
title_tag.text

'A Light in the ...'

In [ ]:
title_tag['title']

'A Light in the Attic'

In [ ]:
title = book_tag.find('a', title=True)['title']
title

'A Light in the Attic'

In [ ]:
rating = book_tag.find('p')['class'][1]
rating

'Three'

In [ ]:
price = book_tag.find('p', class_='price_color').text[1:]
price

'£51.77'

In [ ]:
link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
link


'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [ ]:
def get_details(book_tag):
  title = book_tag.find('a', title=True)['title']
  rating = book_tag.find('p')['class'][1]
  price = book_tag.find('p', class_='price_color').text[1:]
  link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
  return title, rating, price, link

In [ ]:
def get_soup(url):
    """Takes URL and returns a soup object"""
    resp = requests.get(url)
    if resp.status_code == 200:
          return bs(resp.text)
    else: return None
def get_books(url):
    """Extact details from all the book tags"""
    soup=get_soup(url)
    book_tags = soup.find_all('article', class_='product_pod')

    books = []
    for book_tag in book_tags:
        books.append(get_details(book_tag))

    return books

In [ ]:
url = 'http://books.toscrape.com/'
books = get_books(url)
len(books)


20

In [ ]:
books[:3]

[('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('Tipping the Velvet',
  'One',
  '£53.74',
  'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'),
 ('Soumission',
  'One',
  '£50.10',
  'http://books.toscrape.com/catalogue/soumission_998/index.html')]

In [ ]:
import pandas as pd

In [ ]:
def get_all_books(page=3):
  books=[]
  for i in range(1, page+1):
   url = f'http://books.toscrape.com/catalogue/page-{i}.html'
   soup=get_soup(url)
   if soup:
      book_tags = soup.find_all('article', class_='product_pod')

      for book_tag in book_tags:
          books.append(get_details(book_tag))

  books = pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])
  return books

In [ ]:
df = get_all_books(3)
df

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...
5,The Requiem Red,One,£22.65,http://books.toscrape.com/the-requiem-red_995/...
6,The Dirty Little Secrets of Getting Your Dream...,Four,£33.34,http://books.toscrape.com/the-dirty-little-sec...
7,The Coming Woman: A Novel Based on the Life of...,Three,£17.93,http://books.toscrape.com/the-coming-woman-a-n...
8,The Boys in the Boat: Nine Americans and Their...,Four,£22.60,http://books.toscrape.com/the-boys-in-the-boat...
9,The Black Maria,One,£52.15,http://books.toscrape.com/the-black-maria_991/...
